In [ ]:
# Install library yang dibutuhkan
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate pyngrok flask

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

# Inisialisasi model Granite via Replicate
model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),  # Harus sudah diset
    model_kwargs={"max_tokens": 512, "temperature": 0.2},
)


In [ ]:
def zeroshot_prompt_diet(data):
    return f"""
    Saya pengguna aplikasi diet bernama Kaloriku. Hari ini saya telah mengonsumsi {data['kalori_masuk']}kkal dan membakar {data['kalori_terbakar']}kkal melalui aktivitas.
    Target saya adalah defisit {data['target_defisit']}kkal.
    Tolong berikan saran makanan yang harus saya kurangi atau aktivitas tambahan yang bisa saya lakukan agar target defisit tercapai.
    Berikan jawaban ringkas dan mudah dipahami.
    """


In [ ]:
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

@app.route("/saran-ai", methods=["POST"])
def saran_ai():
    data = request.json
    prompt = zeroshot_prompt_diet(data)
    result = model.invoke(prompt)
    return jsonify({"saran": result})

def run_flask():
    app.run(host="0.0.0.0", port=5000)

# Jalankan Flask di thread terpisah
thread = threading.Thread(target=run_flask)
thread.start()


In [ ]:
from pyngrok import ngrok

# GANTI dengan authtoken milikmu
ngrok.set_auth_token("2zLwnLDph938yfIobq2o8Ue8aZS_2GPedgYx8NDYQz3KHv4sV")

# Buat tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


In [ ]:
import requests

data = {
    "kalori_masuk": 1800,
    "kalori_terbakar": 300,
    "target_defisit": 500
}

response = requests.post(f"{public_url}/saran-ai", json=data)
print("Saran AI:\n", response.json()["saran"])
